In [ ]:
#YQL v0
USE hahn;
pragma simplecolumns;
insert into [//tmp/serggor/new_market_vins] with truncate
SELECT a.*, Yson::ConvertToString(Yson::SerializePretty(a.request.request_id)) as request_id
FROM range([//home/voice/vins/logs/dialogs], [2019-10-13], [2019-10-20]) as a

In [117]:
from nile.api.v1 import clusters, Record, filters as nf, aggregators as na

def mapper(records):
    for rec in records:
        if "178278" in rec['testids']:
            for el in rec['session']:
                if  el['intent'] and "market\tmarket" in el['intent']:
                    yield rec
                    break


def mapper1(records):
    for record in records:
        n = len(record['session'])
        first, last = -1, n
        for el in range(n):
            if "market\tmarket" in record['session'][el]['intent'] or "market\tmarket" in record['session'][el].get('restored', ''):
                if first == -1:
                    first = el
                last = el
        first = max(first-3, 0)
        last = min(last+4, n)
        yield Record(record, session=record['session'][first:last])

            
cluster = clusters.YT(proxy="hahn.yt.yandex.net").env(templates={"date": "{2019-10-13..2019-10-20}"})
job = cluster.job()
job.table('//home/voice/dialog/sessions/@date') \
        .map(mapper) \
        .map(mapper1) \
        .random(count=1000) \
        .put('//tmp/serggor/new_market')

job.run()

VBox()

In [118]:
from nile.api.v1 import clusters, Record, filters as nf, aggregators as na

def mapper(records):
    for rec in records:
        for idx, el in enumerate(rec['session']):
            yield Record.from_dict({
                "request_id": el["req_id"],
                "idx": idx,
                "session_id": rec["session_id"]
            })
            
cluster = clusters.YT(proxy="hahn.yt.yandex.net")
job = cluster.job()
job.table('//tmp/serggor/new_market') \
        .map(mapper) \
        .put('//tmp/serggor/new_market_splitted')

job.run()

VBox()

In [ ]:
USE hahn; 
PRAGMA yt.InferSchema;
PRAGMA SimpleColumns;
PRAGMA yt.Pool = "voice";

INSERT INTO [//tmp/serggor/new_market_for_screenshot]
WITH TRUNCATE 
SELECT request_id, session_id, idx, some(mds_key) as mds_key, some(response) as response, some(utterance_text) as utterance_text
FROM (
    SELECT c.request_id as request_id, c.session_id as session_id, c.mds_key as mds_key, c.idx as idx, d.response as response, d.utterance_text as utterance_text
    FROM (
        SELECT some(mds_key) as mds_key, idx, request_id, session_id
        FROM (
            SELECT a.*, b.mds_key as mds_key
            FROM [//tmp/serggor/new_market_splitted] as a
            LEFT JOIN range([//home/voice-speechbase/uniproxy/logs_unified_qloud], [2019-10-13], [2019-10-20]) as b
            ON a.request_id == b.vins_request_id
        )
        GROUP BY idx, request_id, session_id
    ) AS c
    LEFT JOIN [//tmp/serggor/new_market_vins] AS d
    USING(request_id)
)
GROUP BY idx, request_id, session_id

In [ ]:
https://nirvana.yandex-team.ru/flow/29d49660-b250-493d-b622-34c611233455/50ad218e-2301-4f42-bea0-4fda7e0d818e/graph

In [ ]:
PRAGMA yt.InferSchema = '1';
PRAGMA SimpleColumns;
use hahn;
insert into [//tmp/serggor/new_market_screenshots] with truncate 
select request_id, session_id, idx, some(mds_key) as mds_key, some(screenshot_url) as screenshot_url
from (
    select a.idx as idx, a.request_id as request_id, a.session_id as session_id, a.mds_key as mds_key, b.downloadUrl as screenshot_url
    from [//tmp/serggor/new_market_for_screenshot] as a
    left join [//tmp/serggor/market_screenshots] as b
    on a.request_id == substring(b.initialFileName, 12, 36)
) 
group by request_id, session_id, idx

In [123]:
from operator import attrgetter
def mapper(records):
    for record in records:
        rec = record.to_dict()
        if '_other' in rec:
            del rec['_other']
        res = []
        for ses, aud in zip(rec['session'], rec['column']):
            ses['screenshot_url'] = aud['screenshot_url']
            ses['mds_key'] = aud['mds_key']
            res.append(ses)
        rec['session'] = res
        del rec['column']
        yield Record(**rec)
        
        
def reducer(sessions):
    for _, records in sessions:
        t1 = list(map(lambda x: {
            "idx": x["idx"],
            "screenshot_url": x["screenshot_url"],
            "mds_key": x["mds_key"],
            "request_id": x["request_id"],
            "session_id": x["session_id"]
        }, sorted(list(records), key=attrgetter('session_id', 'idx'))))
        t2, cur = [{"session_id": "session_id"}] + t1 + [{"session_id": "session_id"}], []
        for idx in range(1, len(t2)):
            if t2[idx]['session_id'] == t2[idx-1]['session_id']:
                cur.append(t2[idx])
            else:
                if cur:
                    yield Record.from_dict({
                        "column": cur,
                        "session_id": cur[0]["session_id"]
                    })
                cur = [t2[idx]]


cluster = clusters.YT(proxy="hahn.yt.yandex.net")
job = cluster.job()
for_join = job.table("//tmp/serggor/new_market_screenshots") \
        .groupby('session_id') \
        .reduce(reducer)

job.table('//tmp/serggor/new_market') \
        .join(for_join, by='session_id') \
        .map(mapper) \
        .put('//tmp/serggor/new_market_with_all')

job.run()

VBox()

In [ ]:
https://nirvana.yandex-team.ru/flow/29d49660-b250-493d-b622-34c611233455/aa791a3a-b228-4c70-9ed1-7718696ae956/graph